In [179]:
import pandas as pd
import io
import numpy as np
from urllib.request import urlopen
import json
from math import *
from gastrodon import RemoteEndpoint,QName,ttl,URIRef,inline
from matplotlib import pyplot

In [28]:
# helper function to replace None with ''
def none2str(string):
    if string is None:
        return ''
    else:
        return string

In [142]:
# helper to compute distances on the globe
def spherical_dist(pos1, pos2, r=6371000):
    pos1 = pos1 * np.pi / 180
    pos2 = pos2 * np.pi / 180
    cos_lat1 = np.cos(pos1[..., 0])
    cos_lat2 = np.cos(pos2[..., 0])
    cos_lat_d = np.cos(pos1[..., 0] - pos2[..., 0])
    cos_lon_d = np.cos(pos1[..., 1] - pos2[..., 1])
    return r * np.arccos(cos_lat_d - cos_lat1 * cos_lat2 * (1 - cos_lon_d))

In [58]:
#@prefix wikibase: <wikibase: <http://wikiba.se/ontology#> .
prefixes=inline("""
   @prefix wd: <http://www.wikidata.org/entity/> .
   @prefix wdt: <http://www.wikidata.org/prop/direct/> .
   @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
   @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
""").graph
endpoint=RemoteEndpoint(
   "http://query.wikidata.org/sparql"
   ,prefixes=prefixes
)

## Load data

In [2]:
data_url = 'https://data.stadt-zuerich.ch/dataset/brunnen/resource/d741cf9c-63be-495f-8c3e-9418168fcdbf/download/brunnen.json'

In [3]:
data_js = json.loads(urlopen(data_url).read())

## Modify columns

In [62]:
# convert to dataframe
df = pd.io.json.json_normalize(data_js['features'])
# extract coords
df['X'] = df['geometry.coordinates'].apply(lambda crds:crds[0])
df['Y'] = df['geometry.coordinates'].apply(lambda crds:crds[1])
# remove not needed columns
df = df.drop(columns=['geometry.coordinates', 'geometry.type', 'properties.objectid', 'type'])
# rename columns
df = df.rename(index=str, columns=
               {"properties.bezeichnung": "label_de", 
                "properties.brunnenart_txt": "fountain_type", 
                "properties.wasserart_txt": "water_type", 
                "properties.nummer":"operator_id",
                "properties.historisches_baujahr": "date"
               })
# remove "None" from appelation column
df['label_de'] = df['label_de'].apply(lambda a:none2str(a))
df.head()

,label_de,fountain_type,date,operator_id,water_type,X,Y
0,Aussichtsturm,öffentlicher Brunnen,1970.0,510,Verteilnetz,8.599255,47.369752
1,,öffentlicher Brunnen,1933.0,349,Verteilnetz,8.590811,47.369293
2,Biberlinterrasse,öffentlicher Brunnen,1965.0,365,Quellwasser,8.575754,47.366130
3,,öffentlicher Brunnen,1910.0,338,Quellwasser,8.564845,47.370993
4,,Notwasserbrunnen,1988.0,6069,Quellwasser,8.564390,47.369327


## Identify duplicates

In [66]:
# Get bounding box of data
buffer = 0.0003  #(about 20-30 meters)
bounds = {
    'minX': df['X'].min() - buffer,
    'minY': df['Y'].min() - buffer,
    'maxX': df['X'].max() + buffer,
    'maxY': df['Y'].max() + buffer
}

In [78]:
# query fountains within bounding box
query_string = """ SELECT ?place ?placeLabel ?location
WHERE
{{
  # Enter coordinates
  SERVICE wikibase:box {{
    ?place wdt:P625 ?location .
    bd:serviceParam wikibase:cornerWest "Point({minX} {minY})"^^geo:wktLiteral.
    bd:serviceParam wikibase:cornerEast "Point({maxX} {maxY})"^^geo:wktLiteral.
  }} .
  # Is a water well or fountain or subclass of fountain
  FILTER (EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q43483 }} || EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q483453 }}).
  SERVICE wikibase:label {{
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" .
  }} 
}}
  """.format(**bounds)

# Perform query
query_result = endpoint.select(query)

In [81]:
# Extract lat and lon
query_result['X'] = query_result['location'].apply(lambda l:float(l.split('(')[1].split(' ')[0]))
query_result['Y'] = query_result['location'].apply(lambda l:float(l.split(' ')[1].split(')')[0]))

In [151]:
# compute distance matrix
distances = spherical_dist(df[['X','Y']].values[:, None], query_result[['X','Y']].values)

# find nearest existing fountain for each fountain to import
#indexes of nearest
nearest_idx = np.argmin(distances, axis=1).tolist()
# qid of nearest
df['nearest_qid'] = query_result.iloc[nearest_idx]['place'].apply(lambda id:id[3:]).tolist()
# distance of nearest
df['nearest_distance'] = np.min(distances, axis=1).tolist()


# then remove nearest
i_line=0
for i_col in nearest_idx:
    distances[i_line, i_col] = 100000
    i_line += 1
# find distance to second nearest
df['2nd_nearest_distance'] = np.min(distances, axis=1).tolist()

df.head()

,label_de,fountain_type,date,operator_id,water_type,X,Y,nearest_qid,nearest_distance,2nd_nearest_distance
0,Aussichtsturm,öffentlicher Brunnen,1970.0,510,Verteilnetz,8.599255,47.369752,Q27229822,1572.747666,1713.116545
1,,öffentlicher Brunnen,1933.0,349,Verteilnetz,8.590811,47.369293,Q27229822,1175.673373,1485.271348
2,Biberlinterrasse,öffentlicher Brunnen,1965.0,365,Quellwasser,8.575754,47.366130,Q27230070,456.499616,634.066324
3,,öffentlicher Brunnen,1910.0,338,Quellwasser,8.564845,47.370993,Q27230192,1.980033,226.992084
4,,Notwasserbrunnen,1988.0,6069,Quellwasser,8.564390,47.369327,Q27230192,188.767471,284.923890


In [156]:
# use the identified qid only if criteria are met: 
# - no further than 10 m away
# - next closest fountain at neares least 50% further away
def validate_proposal(qid, d1, d2, dmax=10, ratio_min=0.5):
    
    if d1 == 0 or (d1<=dmax and d2/d1-1 >= ratio_min):
        return qid
    else:
        return ''

In [167]:
for index, row in df.iterrows():
    df.loc[index, 'qid'] = validate_proposal(
        row['nearest_qid'], 
        row['nearest_distance'], 
        row['2nd_nearest_distance'],
        dmax=15
    )
dffinal = df.drop(columns=['nearest_qid', 'nearest_distance', '2nd_nearest_distance'])

In [199]:
dffinal.head()

,label_de,fountain_type,date,operator_id,water_type,X,Y,qid
0,Aussichtsturm,öffentlicher Brunnen,1970.0,510,Verteilnetz,8.599255,47.369752,
1,,öffentlicher Brunnen,1933.0,349,Verteilnetz,8.590811,47.369293,
2,Biberlinterrasse,öffentlicher Brunnen,1965.0,365,Quellwasser,8.575754,47.366130,
3,,öffentlicher Brunnen,1910.0,338,Quellwasser,8.564845,47.370993,Q27230192
4,,Notwasserbrunnen,1988.0,6069,Quellwasser,8.564390,47.369327,


## Format for wikidata import

In [196]:
def process_coordinates(x, y):
    return '@{1:1.8f}/{0:1.8f}'.format(x,y)

def process_year(date):
    if np.isnan(date):
        return ''
    else:
        return '+{0:4d}-00-00T00:00:00Z/9.'.format(int(date))

fountain_type_map = {
    'öffentlicher Brunnen': 'Q53628296',
    'Notwasserbrunnen': 'Q53628522',
    'privater Brunnen': 'Q53629707',
    'Brunnen in städtischer Liegenschaft': 'Q53628618',
    'Brunnen des Verschönerungsvereins': 'Q53628761',
    'Brunnen mit eigener Versorgung': 'Q53630002'
}

def process_fountain_type(type):
    return fountain_type_map[type]

def createline(lines, item, prop, value, qualifiers=[]):
    if value != '':
        statement = '{}\t{}\t{}'.format(item, prop, value)
        if len(qualifiers):
            # append qualifiers if applicable
            for q in qualifiers:
                statement += '\t{}\t{}'.format(q['prop'], q['value'])
        statement += '\n'
        lines.append(statement)
    return lines

In [201]:
lines = []

for index, row in dffinal.iterrows():
    if row['qid'] == '':
        # create a new fountain
        lines.append('CREATE\n')
        item = 'LAST'
    else:
        # update existing fountain
        item = row['qid']
        
    # label in german
    lines = createline(lines, item, 'Lde', '"{}"'.format(row['label_de']))
    
    # instance of drinking fountain
    lines = createline(lines, item, 'P31', 'Q1630622')
    
    # instance of specific water fountain type
    lines = createline(lines, item, 'P31', process_fountain_type(row['fountain_type']))
    
    # coordinates
    lines = createline(lines, item, 'P625', process_coordinates(row['X'], row['Y']))
    
    # creation date
    lines = createline(lines, item, 'P571', process_year(row['date']))
    
    # operated by WVZ
    lines = createline(lines, item, 'P137', 'Q27229237')
    
    # catalog number
    lines = createline(lines, item, 'P528', row['operator_id'], [{
        'prop': 'P972',
        'value': 'Q53629101'
    }])

# Write commands to file

In [202]:
with io.open("quickstatement_commands.txt", "w", encoding='utf8') as f:
    f.writelines(lines)